# バックアップ＆リストア
----
Data&nbsp;Node上にあるデータのバックアップと リストアについて説明します。


**<font color="red">このデモ環境では実施できません。クラスタを運用した場合のイメージとしてお読みください。</font>**

## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [53]:
%env ES_HOST=XXX.XXX.XXX.233:9200

env: ES_HOST=XXX.XXX.XXX.233:9200


また、データを蓄積する先のインデックスとタイプを、次のセルを実行して保存してください。  

※特にセルの内容の書き換えは必要ありません。  
　前の章と異なるインデックス名にしているのは、この章のコマンドで投入する内容が前の章までのものと混ざらないようにするためです。

In [1]:
%env INDEX=meteorological-data-*

env: INDEX=meteorological-data-*


## バックアップ
バックアップ方法について説明します。

Elasticsearchのバックアップはクラスタ・index単位で取得することができます。   
詳細はElasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

### バックアップ対象indexの容量確認
バックアップの保存前に、対象となるindexが保存先のストレージに収まる容量かを確認します。

indexの容量はcat APIにより確認できます。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/cat-indices.html)を参照してください。

In [80]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb
green  open   reindex_source FTQ0QcHxR9qVpt_sART-YQ   5   1          1            0      8.9kb          4.4kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   330  100   330    0     0  24422      0 --:--:-- --:--:-- --:--:-- 25384


### リポジトリの作成（初回のみ）
保存先であるリポジトリを作成します。

Elasticsearchでバックアップを保存するには、データのスナップショットを保存する先であるリポジトリを作成する必要があります。  
my_backupという名前のリポジトリを作るコマンドは次の通りです。

In [56]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup" -d @- << EOF
{
    "type": "fs",
    "settings": {
        "location": "/mnt/backup/my_backup",
        "compress": true
    }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   130  100    21  100   109    229   1193 --:--:-- --:--:-- --:--:--  1197


my_backupリポジトリが作成されたことを確認します

In [57]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup?pretty"

{
  "my_backup" : {
    "type" : "fs",
    "settings" : {
      "compress" : "true",
      "location" : "/mnt/backup/my_backup"
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100   140    0     0   8829      0 --:--:-- --:--:-- --:--:--  9333


### バックアップ実行
実際にデータをバックアップします。  
クラスタ全体で取るのか、index単位で取るのかでコマンドを切り替えてください。


オプションについては、Elasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

**■クラスタ全体をバックアップ**

クラスタ全体のバックアップを、my_backupリポジトリにsnapshot_1という名前で保存するコマンドは次の通りです。

In [58]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_1?wait_for_completion=true&pretty"

{
  "snapshot" : {
    "snapshot" : "snapshot_1",
    "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
    "version_id" : 5000099,
    "version" : "5.0.0",
    "indices" : [
      "sample_index",
      "meteorological-data-2015.07",
      "meteorological-data-2015.08",
      "meteorological-data-2015.03",
      "meteorological-data-2015.09",
      "meteorological-data-2015.05",
      "meteorological-data-2015.01",
      "meteorological-data-2015.04",
      "meteorological-data-2015.12",
      "meteorological-data-2015.11",
      "meteorological-data-2015.10",
      "meteorological-data-2015.02",
      "meteorological-data-2015.06",
      "meteorological-data-2014.12"
    ],
    "state" : "SUCCESS",
    "start_time" : "2016-12-13T04:31:41.470Z",
    "start_time_in_millis" : 1481603501470,
    "end_time" : "2016-12-13T04:31:42.079Z",
    "end_time_in_millis" : 1481603502079,
    "duration_in_millis" : 609,
    "failures" : [ ],
    "shards" : {
      "total" : 70,
      "failed" : 0,
      "successful

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1019  100  1019    0     0   1555      0 --:--:-- --:--:-- --:--:--  1555


**■インデックスを指定してバックアップ**

meteorological-data-\* indexのバックアップを、my_backupリポジトリにsnapshot_1という名前で保存するコマンドは次の通りです。

In [71]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_1" -d @- << EOF
{
  "indices": "$INDEX",
  "ignore_unavailable": true,
  "include_global_state": false
}
EOF

{"accepted":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   116  100    17  100    99    153    891 --:--:-- --:--:-- --:--:--   900


### バックアップできたかの確認
リポジトリの内容を閲覧して、バックアップが取れたかを確認します。

snapshot_1という名前のバックアップ内容を確認するコマンドは次の通りです。

In [59]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup/snapshot_1?pretty"

{
  "snapshots" : [
    {
      "snapshot" : "snapshot_1",
      "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
      "version_id" : 5000099,
      "version" : "5.0.0",
      "indices" : [
        "sample_index",
        "meteorological-data-2015.07",
        "meteorological-data-2015.08",
        "meteorological-data-2015.03",
        "meteorological-data-2015.09",
        "meteorological-data-2015.05",
        "meteorological-data-2015.01",
        "meteorological-data-2015.04",
        "meteorological-data-2015.12",
        "meteorological-data-2015.11",
        "meteorological-data-2015.10",
        "meteorological-data-2015.02",
        "meteorological-data-2015.06",
        "meteorological-data-2014.12"
      ],
      "state" : "SUCCESS",
      "start_time" : "2016-12-13T04:31:41.470Z",
      "start_time_in_millis" : 1481603501470,
      "end_time" : "2016-12-13T04:31:42.079Z",
      "end_time_in_millis" : 1481603502079,
      "duration_in_millis" : 609,
      "failures" : [ ],
      "shards

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1096  100  1096    0     0  72823      0 --:--:-- --:--:-- --:--:-- 78285


## リストア
リストア方法について説明します。

### リストア先のindex状況の確認
データリストアはindexが空のところに入れることを前提としています。  
リストア実施前に、同名indexが存在しないことを確認してください。 

現状存在するindexを確認するコマンドは、次の通りです。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/cat-indices.html)を参照してください。

In [81]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   reindex_dest   hm0itWDWTP-h05dMnJSRAQ   5   1          1            0      8.8kb          4.4kb
green  open   reindex_source 0CVwNJ0XQievecjHPq5kNQ   5   1          1            0      8.9kb          4.4kb
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   440  100   440    0     0  28299      0 --:--:-- --:--:-- --:--:-- 29333


また、snapshotから一部のindexをリストアすることやindex名を変更しながらリストアすることも可能ですので、  
状況に応じて必要なindexをリストアしてください。    
詳細はElasticsearch Referenceの[Restoring from a Snapshotedit](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

### リストア実行
実際にデータをリストアします。

例として、[バックアップ実行](#バックアップ実行)で取得したバックアップ(snapshot_1)からリストアするコマンドは次の通りです。

In [77]:
%%bash
curl -XPOST "http://$ES_HOST/_snapshot/my_backup/snapshot_1/_restore?pretty"

{
  "accepted" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24  100    24    0     0    239      0 --:--:-- --:--:-- --:--:--   242
